In [1]:
import numpy as np
import igl
import meshplot as mp
import math as math
import sympy as sp

In C:\Users\Leeovn\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Leeovn\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Leeovn\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Leeovn\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Leeovn\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)

    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/luigi.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

In [4]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [5]:
pi

array([[  0. ,   0. ,   0. ],
       [ -0.1,  -6. ,   1. ],
       [ -4.4,  -3.4,  -1. ],
       ...,
       [ 26.4, -37.2, -96.2],
       [ 26.2, -44.4, -96.1],
       [ 24.8, -50.2, -96.4]])

# MLS function

In [6]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [7]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [8]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [9]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Setting up the Constraints

In [10]:
#Calucalte the min and max for each point

max_x, max_y, max_z = min_x, min_y, min_z = pi[0][0], pi[0][1], pi[0][2]

for i in range(pi.shape[0]):
    
    if (pi[i][0] > max_x):
        max_x = pi[i][0]
    if (pi[i][0] < min_x):
        min_x = pi[i][0]
        
    if (pi[i][1] > max_y):
        max_y = pi[i][1]
    if (pi[i][1] < min_y):
        min_y = pi[i][1]
    if (pi[i][2] > max_z):
        max_z = pi[i][2]
    if (pi[i][2] < min_z):
        min_z = pi[i][2]
# Add buffer
max_x += 5
min_x -= 5
max_y += 5
min_y -= 5
max_z += 5
min_z -= 20

In [11]:
(min_x, min_y, min_z)

(-27.7, -68.7, -119.0)

In [12]:
# Create a grid sampling the 3D space
# Parameters
bbox_min = np.array([min_x, min_y, min_z])
bbox_max = np.array([max_x, max_y, max_z])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)
eps = 0.01 * bbox_diag;
bbox_min

array([ -27.7,  -68.7, -119. ])

In [13]:
pi = pi.copy()
ni = ni.copy()
eps = 0.005 * bbox_diag;
pi, pi.shape

(array([[  0. ,   0. ,   0. ],
        [ -0.1,  -6. ,   1. ],
        [ -4.4,  -3.4,  -1. ],
        ...,
        [ 26.4, -37.2, -96.2],
        [ 26.2, -44.4, -96.1],
        [ 24.8, -50.2, -96.4]]), (366, 3))

In [14]:
bbox_diag

171.164715990183

In [15]:
eps

0.855823579950915

In [16]:
def find_closest_point(point, points):
    '''Retreives the index of the closest point in points'''
    index = -1;
    min_dist = 0;

    for i in range (points.shape[0]):
        if ((points[i] == point).all()):
            continue
        
        dist = np.linalg.norm(point-points[i]);
        if (min_dist == 0 or dist < min_dist):
            min_dist = dist
            index = i
    
    assert(index != -1)
    return index;

In [17]:
# Test, solution should be 6
points = np.zeros((10, 3))
for i in range (points.shape[0]):
    points[i] = [4 - i, 3 * i, i]
points[1] = [50, 3, 1]
points[6] = [3, 0, 1]
points
find_closest_point(points[0], points)

6

In [18]:
def find_eps(index, point, normal, points, eps, plus=True):
    # return pi_plus if plus= True, else return pi_minus
    #print("test")
    if (plus):
        pi_eps = np.array([point + eps * normal])
    else:
        pi_eps = np.array([point - eps * normal])
    
    tmp_pi = np.concatenate((points, pi_eps))
    if (index == find_closest_point(tmp_pi[pi.shape[0]], tmp_pi)):
        return (pi_eps, eps)
    return (find_eps(index, point, normal, points, eps/2.0, plus))

In [19]:
# For each point pi in the point cloud, add a constraint of the form f(pi) = di = 0
# Fix an eps value, for instance eps = 0.01 * bounding_box_diagonal
# For each point pi compute pi+ = pi + eps * ni, where ni is the normalized normal of pi. 
# Check that pi is the closest point to pi+, if not, halve esp and recompute pi+ until this is the case. Then, add another constraint
# equation: f(pi+) = eps
# Repeat this same process for -eps,
# Final result is a unqiue vector or pi, pi+, and pi- to p and corresponding f
p = np.zeros((pi.shape[0] * 3, 3))
f = np.zeros(pi.shape[0]*3)
color = np.zeros((pi.shape[0]*3, 3))

for i in range(pi.shape[0]):
    p[3*i] = pi[i]
    f[3*i] = 0
    color[3*i] = [0.0, 0.0, 1.0]
    
    pi_plus, found_eps = find_eps(i, pi[i], ni[i], pi, eps, True)
    p[3*i + 1] = pi_plus
    f[3*i + 1] = found_eps
    color[3*i + 1] = [1.0, 0.0, 0.0]
        
    pi_minus, found_eps = find_eps(i, pi[i], ni[i], pi, eps, False)
    p[3*i + 2] = pi_minus
    f[3*i + 2] = -found_eps
    color[3*i + 2] = [0.0, 1.0, 0.0]

In [20]:
mp.plot(p, c=color, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0072269…

# Use MLS interpolation to extend to function f

In [21]:
n = 25

In [22]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

In [23]:
x

array([[ -36.2582358 ,  -77.2582358 , -127.5582358 ],
       [ -36.2582358 ,  -77.2582358 , -121.43671615],
       [ -36.2582358 ,  -77.2582358 , -115.3151965 ],
       ...,
       [  46.2582358 ,   30.2582358 ,    7.1151965 ],
       [  46.2582358 ,   30.2582358 ,   13.23671615],
       [  46.2582358 ,   30.2582358 ,   19.3582358 ]])

In [24]:
A = np.array([[0,1,2], [3,4,5], [6,7,8], [9,10,11]])
A_T = np.transpose(A)
A_T, A_T.shape

(array([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]]), (3, 4))

In [25]:
def closest_points(point, points, h):
    '''Retrieves the indices of all points in points that are at distance less than h from point'''
    indices = []
    for i in range(points.shape[0]):       
        dist = np.linalg.norm(point-points[i]);
        if (dist < h):
           #print("points p[", i, "]:", points[i], "dist:", dist)
            indices.append(i)
    #if (len(indices) > 0):
        #print("indices:", indices)
    return indices

In [26]:
def wendlandFunction(r, h):
    return pow((1 - r/h), 4) * (4*r/h + 1)

In [27]:
def create_b_j(pi, degPoly):
    if (degPoly == 0):
        return np.array([1])
    if (degPoly == 1):
        return np.array([1.0, pi[0], pi[1], pi[2]])
    if (degPoly == 2):
        return np.array([1.0, pi[0], pi[1], pi[2], pi[0]*pi[1], pi[0]*pi[2], pi[1]*pi[2], pi[0]*pi[0], pi[1]*pi[1], pi[2]*pi[2]])
    print('Error: wrong degPoly parm given. Must be within [0, 2]')
    return None

In [28]:
# Solve for p_xi (points p with norm(xi - p) < wedlandRadius)
h = 25
p_x = []
for i in range(x.shape[0]):
    #print("point:x[", i, "]:", x[i], "points: p:")
    indices = closest_points(x[i], p, h)
    #if(len(indices) > 0):
        #print("Appending indices to p_x[", i, "]")
    p_x.append(indices)
#print(p_x)

In [29]:
fx = np.zeros((x.shape[0]))
degPoly = 0;
if (degPoly == 1):
    numPoly = 4
elif (degPoly == 2):
    numPoly = 10
else:
    numPoly = 1
    

for i in range(x.shape[0]):
    # for each grid point x
    if (len(p_x[i]) < numPoly*2):
        # num of constraint points is less than twice the number of polynomial coefficients
        fx[i] = 100
    else:
        # solve fx[i] = (float) sum over j bj * aj
        # find (numPoly x 1) matrix a using Aa = B'
        # where  A  = (B^T * W * B) is a numPoly x numPoly matrix
        # and    B' = (B^T * W * di) is a numPoly x 1 matrix
        # and    B  = len(p_x[i]) x numPoly matrix
        # and    W  = len(p_x[i]) x len(p_x[i]) matrix
        # and    di = len(p_x[i]) x 1 matrix
        # finally a = numPoly x 1 matrix
        B = np.zeros((len(p_x[i]), numPoly))
        W = np.zeros((len(p_x[i]), len(p_x[i])))
        di = np.zeros((len(p_x[i]), 1))
        
        for j in range(len(p_x[i])):
            r = np.linalg.norm(x[i] - p[p_x[i][j]])
            W[j][j] = wendlandFunction(r, h)
            B[j] = create_b_j(p[p_x[i][j]], degPoly)
            di[j] = f[p_x[i][j]]
        A = np.dot(np.dot(np.transpose(B), W), B)
        B_prime = np.dot(np.dot(np.transpose(B), W), di)
        a = np.linalg.solve(A, B_prime)
        #print("len(p_x[", i, "]):", len(p_x[i]), "degPoly:", degPoly)
        #print("W shape:", W.shape, "B shape:", B.shape, "di shape:", di.shape, "A shape:", A.shape, "B' shape:", B_prime.shape, "a shape:", a.shape)
        for j in range(len(p_x[i])):
            fx[i] += np.dot(B[j], a)

In [30]:
fx

array([100., 100., 100., ..., 100., 100., 100.])

In [31]:
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 8,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Extracting the surface

In [32]:
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.3394727…

# Implementing a spatial index to accelerate neighbor calculations

### This section didn't plan out like expected. Unfortunately I am a little too exhausted to get this over the finish line.

In [36]:
grid_n = 10 # i, j, k domain = [0, 10] -> 10 x 10 grid
grid = [[[[] for k in range(grid_n+1)] for j in range(grid_n+1)] for i in range(grid_n+1)]
#print(grid)
max_size = round(max(max_x, max_y, max_z)/10)*10 # max size of the bbox rounded to nearest 10
min_pos = round((min(min_x, min_y, min_z)-20)/10)*10 # min position the bbox approaches from x, y, z
w = (max_size-min_pos)/grid_n # determine distance between equally subdived cell walls
w, min_pos
print(w)

for index in range(p.shape[0]):
    i = math.floor((p[index][0] - min_pos)/w)
    j = math.floor((p[index][1] - min_pos)/w)
    k = math.floor((p[index][2] - min_pos)/w)
    grid[i][j][k].append(index)

#print(grid)

15.0


In [ ]:
fx = np.zeros((x.shape[0]))
degPoly = 0;
if (degPoly == 1):
    numPoly = 4
elif (degPoly == 2):
    numPoly = 10
else:
    numPoly = 1
radius = [-h, 0, h];

for index in range(x.shape[0]):
    cell_indices = []
    prev_cell_indices = []
    
    #print("pos:", x[index][0], x[index][1], x[index][2])
    curr_i = math.floor((x[index][0] - min_pos)/w)
    curr_j = math.floor((x[index][1] - min_pos)/w)
    curr_k = math.floor((x[index][2] - min_pos)/w)
    #print("x[", index, "]:", x[index], "CURRR:", curr_i, curr_j, curr_k)
    grid_x = grid[curr_i][curr_j][curr_k]
    #print("i:", curr_i, "j:", curr_j, "k:", curr_k, "grid_x:", grid_x)
    
    for i in range(len(radius)):
        for j in range(len(radius)):
            for k in range(len(radius)):
                cell_i = math.floor((x[index][0]+radius[i] - min_pos)/w)
                cell_j = math.floor((x[index][1]+radius[j] - min_pos)/w)
                cell_k = math.floor((x[index][2]+radius[k] - min_pos)/w)
                if (([cell_i, cell_j, cell_k] in prev_cell_indices) 
                    or (cell_i == -1 or cell_j == -1 or cell_k == -1)) or (cell_i == grid_n+1 or cell_j == grid_n+1 or cell_k == grid_n+1):
                    continue
                else:
                    #print("x[", index, "]:", x[index], "cell:", cell_i, cell_j, cell_k, "TEST:", cell_indices)
                    cell_indices.extend(grid[cell_i][cell_j][cell_k])
                    prev_cell_indices.append([cell_i, cell_j, cell_k])

    if (len(cell_indices) < numPoly*2):
        fx[index] = 100
    else:
        # solve fx[i] = (float) sum over j bj * aj
        # find (numPoly x 1) matrix a using Aa = B'
        # where  A  = (B^T * W * B) is a numPoly x numPoly matrix
        # and    B' = (B^T * W * di) is a numPoly x 1 matrix
        # and    B  = len(cell_indices) x numPoly matrix
        # and    W  = len(cell_indices) x len(cell_indices) matrix
        # and    di = len(cell_indices) x 1 matrix
        # finally a = numPoly x 1 matrix
        B = np.zeros((len(cell_indices), numPoly))
        W = np.zeros((len(cell_indices), len(cell_indices)))
        di = np.zeros((len(cell_indices), 1))
        for pi in range(len(cell_indices)):
            r = np.linalg.norm(x[index] - p[cell_indices[pi]])
            W[pi][pi] = wendlandFunction(r, h)
            B[pi] = create_b_j(p[cell_indices[pi]], degPoly)
            di[pi] = f[cell_indices[pi]]
        A = np.dot(np.dot(np.transpose(B), W), B)
        B_prime = np.dot(np.dot(np.transpose(B), W), di)
        a = np.linalg.solve(A, B_prime)
        #print("len(p_x[", i, "]):", len(p_x[i]), "degPoly:", degPoly)
        #print("W shape:", W.shape, "B shape:", B.shape, "di shape:", di.shape, "A shape:", A.shape, "B' shape:", B_prime.shape, "a shape:", a.shape)
        for j in range(len(cell_indices)):
            fx[index] += np.dot(B[j], a)

In [ ]:
fx

In [36]:
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 8,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Using a non-axis-aligned grid.

In [41]:
pi, v = igl.read_triangle_mesh("data/luigi.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

In [42]:
Y = pi-np.mean(pi, axis=0)
S=Y.T@Y

l,v =np.linalg.eig(S)

p=mp.plot(Y,shading={"point_size": 0.5})
p.add_lines(np.array([0,0,0]), v[:, 0]*10)
p.add_lines(np.array([0,0,0]), v[:, 1]*10)
p.add_lines(np.array([0,0,0]), v[:, 2]*10)
v, l, 
S

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.009680…

array([[14365.56275095, 11665.95411589,  7657.9574822 ],
       [11665.95411589, 15989.37373242, 11280.53693744],
       [ 7657.9574822 , 11280.53693744, 13911.63246754]])

In [43]:
pi = pi.copy()
ni = ni.copy()
eps = 0.01 * bbox_diag;
pi, pi.shape

(array([[-1.5493 ,  1.429  ,  0.35299],
        [ 2.7288 ,  2.9492 , -2.1646 ],
        [-2.7832 , -0.67427, -3.9072 ],
        ...,
        [ 2.2225 ,  2.668  , -2.8415 ],
        [ 2.3444 ,  6.0958 ,  1.7899 ],
        [-0.52428,  1.3474 , -2.7328 ]]), (2502, 3))

In [44]:
for i in range(len(pi)):
    pi[i][0] *= v[0, 0]
    pi[i][1] *= v[1, 1]
    pi[i][2] *= v[2, 2]
    ni[i][0] *= v[0, 0]
    ni[i][1] *= v[1, 1]
    ni[i][2] *= v[2, 2]

In [45]:
pi

array([[ 0.85326227, -0.00598313,  0.15626195],
       [-1.5028607 , -0.0123481 , -0.9582272 ],
       [ 1.53282098,  0.00282312, -1.72964304],
       ...,
       [-1.22402078, -0.01117073, -1.25787794],
       [-1.29115604, -0.0255227 ,  0.79235464],
       [ 0.28874223, -0.00564147, -1.20975852]])

In [46]:
#Calucalte the min and max for each point

max_x, max_y, max_z = min_x, min_y, min_z = pi[0][0], pi[0][1], pi[0][2]

for i in range(pi.shape[0]):
    
    if (pi[i][0] > max_x):
        max_x = pi[i][0]
    if (pi[i][0] < min_x):
        min_x = pi[i][0]
        
    if (pi[i][1] > max_y):
        max_y = pi[i][1]
    if (pi[i][1] < min_y):
        min_y = pi[i][1]
    if (pi[i][2] > max_z):
        max_z = pi[i][2]
    if (pi[i][2] < min_z):
        min_z = pi[i][2]
# Add buffer
max_x += 5
min_x -= 5
max_y += 5
min_y -= 5
max_z += 5
min_z -= 20

In [ ]:
# For each point pi in the point cloud, add a constraint of the form f(pi) = di = 0
# Fix an eps value, for instance eps = 0.01 * bounding_box_diagonal
# For each point pi compute pi+ = pi + eps * ni, where ni is the normalized normal of pi. 
# Check that pi is the closest point to pi+, if not, halve esp and recompute pi+ until this is the case. Then, add another constraint
# equation: f(pi+) = eps
# Repeat this same process for -eps,
# Final result is a unqiue vector or pi, pi+, and pi- to p and corresponding f
p = np.zeros((pi.shape[0] * 3, 3))
f = np.zeros(pi.shape[0]*3)
color = np.zeros((pi.shape[0]*3, 3))

for i in range(pi.shape[0]):
    p[3*i] = pi[i]
    f[3*i] = 0
    color[3*i] = [0.0, 0.0, 1.0]
    
    pi_plus, found_eps = find_eps(i, pi[i], ni[i], pi, eps, True)
    p[3*i + 1] = pi_plus
    f[3*i + 1] = found_eps
    color[3*i + 1] = [1.0, 0.0, 0.0]
        
    pi_minus, found_eps = find_eps(i, pi[i], ni[i], pi, eps, False)
    p[3*i + 2] = pi_minus
    f[3*i + 2] = -found_eps
    color[3*i + 2] = [0.0, 1.0, 0.0]
    

In [ ]:
mp.plot(p, c=color, shading={"point_size": 0.1})
p.add_lines(np.array([0,0,0]), v[:, 0]*10)
p.add_lines(np.array([0,0,0]), v[:, 1]*10)
p.add_lines(np.array([0,0,0]), v[:, 2]*10)